# Recommending Locations for an Italian Restaurant in Fort Worth

In [ ]:
# package imports
import requests
import kaggle
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup

## Data Collection

### Demographics

In [41]:
# API Endpoints for Fort Worth and National Data
fort_worth_url = "https://api.censusreporter.org/1.0/data/show/latest?geo_ids=16000US4827000&table_ids=B01003,B19013,B01002"
us_url = "https://api.censusreporter.org/1.0/data/show/latest?geo_ids=01000US&table_ids=B01003,B19013,B01002"

# Fetch data
fort_worth_data = requests.get(fort_worth_url).json()
us_data = requests.get(us_url).json()

# Extract key statistics
fort_worth_geo = fort_worth_data["data"]["16000US4827000"]
us_geo = us_data["data"]["01000US"]

# Land area values (Census QuickFacts)
fort_worth_land_area = 342.2  # Fort Worth land area (sq. miles)
us_land_area = 3531905  # US total land area (sq. miles)

# Calculate population density
fort_worth_population = fort_worth_geo["B01003"]["estimate"]["B01003001"]
us_population = us_geo["B01003"]["estimate"]["B01003001"]

fort_worth_density = fort_worth_population / fort_worth_land_area
us_density = us_population / us_land_area

# Convert to DataFrame
df = pd.DataFrame({
    "Metric": ["Median Household Income", "Median Age", "Population Density (per sq. mile)"],
    "Fort Worth": [
        fort_worth_geo["B19013"]["estimate"]["B19013001"],  # Fort Worth Median Income
        fort_worth_geo["B01002"]["estimate"]["B01002001"],  # Fort Worth Median Age
        round(fort_worth_density, 2)  # Fort Worth Population Density
    ],
    "National Average": [
        us_geo["B19013"]["estimate"]["B19013001"],  # US Median Income
        us_geo["B01002"]["estimate"]["B01002001"],  # US Median Age
        round(us_density, 2)  # US Population Density
    ]
})

# Display comparison table
print(df)

# Explain the comparison
print("\n{0} is {1:.2f} times than the US medium household income.".format("Fort Worth", fort_worth_geo["B19013"]["estimate"]["B19013001"] / us_geo["B19013"]["estimate"]["B19013001"]))
print("{0} is {1:.2f} younger than the US median age.".format("Fort Worth", abs(fort_worth_geo["B01002"]["estimate"]["B01002001"] - us_geo["B01002"]["estimate"]["B01002001"])))
print("{0} is {1:.2f} times denser than the US average.".format("Fort Worth", fort_worth_density / us_density))


                              Metric  Fort Worth  National Average
0            Median Household Income    77082.00          77719.00
1                         Median Age       33.70             39.20
2  Population Density (per sq. mile)     2854.86             94.83

Fort Worth is 0.99 times than the US medium household income.
Fort Worth is 5.50 younger than the US median age.
Fort Worth is 30.11 times denser than the US average.


From the Census demographic data, we can determine the fort worth is a urban city with a younger population. The household income in this city is similar to the national average. This will be the target demographic when exploring possible locations for a resturant. 

### Competitive Landscape

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Set up Kaggle API key (ensure kaggle.json is in the correct directory)
os.environ['KAGGLE_CONFIG_DIR'] = "/path_to_kaggle_json_folder"  # Adjust the path to where your kaggle.json is located

# Initialize the Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset (replace 'shrutimehta/zomato-restaurants-data' with the dataset ID)
api.dataset_download_files('shrutimehta/zomato-restaurants-data', path='./', unzip=True)


### Accessibility & Foot Traffic


### Crime & Safety Metrics

In [44]:
import requests
from bs4 import BeautifulSoup

# Define the URL to scrape
url = "https://www.neighborhoodscout.com/tx/fort-worth/crime"

# Send a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Scrape the "Safest Fort Worth Area Neighborhoods" section
    safest_neighborhoods_section = soup.find("section", {"id": "Safest Fort Worth neighborhoods"})
    safest_neighborhoods = safest_neighborhoods_section.find_all("li") if safest_neighborhoods_section else []
    
    safest_neighborhoods_list = [neighborhood.get_text() for neighborhood in safest_neighborhoods]
    
    # Scrape the "Fort Worth Crime Data" section
    crime_data_section = soup.find("section", {"id": "Fort Worth Crime"})
    crime_data_headers = crime_data_section.find_all("h2") if crime_data_section else []
    crime_data_paragraphs = crime_data_section.find_all("p") if crime_data_section else []
    
    crime_data_headers_list = [header.get_text() for header in crime_data_headers]
    crime_data_paragraphs_list = [paragraph.get_text() for paragraph in crime_data_paragraphs]
    
    # Print the scraped data for review
    print("Safest Fort Worth Area Neighborhoods:")
    for neighborhood in safest_neighborhoods_list:
        print(neighborhood)
    
    print("\nFort Worth Crime Data:")
    for header in crime_data_headers_list:
        print(header)
    
    for paragraph in crime_data_paragraphs_list:
        print(paragraph)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Safest Fort Worth Area Neighborhoods:

Fort Worth Crime Data:


In [45]:
# Example: Assuming the crime data is structured in a specific way
crime_data = []

# Example of extracting data from specific tags (adjust according to the structure)
for para in paragraphs:
    crime_data.append(para.get_text())

# Convert the data into a pandas DataFrame for better readability
df = pd.DataFrame(crime_data, columns=["Crime Data"])
df.head()  # Display the first few rows


,Crime Data
0,"\nAnalytics built by:\n \nLocation, Inc.\n"
1,"\nRaw data sources:\n \n18,000 local law enfor..."
2,"\nDate(s) & Update Frequency:\n \n""Reflects 20..."
3,\nMethodology:\n \nOur nationwide meta-analysi...
4,With a crime rate of 31 per one thousand resid...
